In [1]:
import pandas as pd
import os
import glob

In [2]:
# Working directory
project_dir = '/zfs/projects/darc/wolee_edehaan_suzienoh-exploratory-ml'
os.chdir(project_dir)

In [3]:
def get_prediction_file_paths(directory):
    # Use glob to get all csv files in the directory
    csv_files = glob.glob(os.path.join(directory, '*prediction*.csv'))
    return csv_files

In [4]:
def postprocess_predictions(prediction_df, prediction_col='pred', period='month'):
    
    # Target name based on period
    if period == 'quarter':
        target = 'retq'
    elif period == 'month':
        target = 'ret'
    else:
        raise ValueError("period must be 'quarter' or 'month'")
        
    prediction_df['prob']=prediction_df[prediction_col]
    prediction_df.sort_values('prob', inplace=True)
    prediction_df['rank'] = prediction_df.groupby(['date'])['prob'].transform(lambda x: pd.qcut(x.values, 10, labels=False, duplicates='drop'))
    prediction_df['port_size'] = prediction_df.groupby(['date','rank'])['mve_m'].transform('sum')
    prediction_df['port_ret'] = prediction_df[target] * prediction_df['mve_m']/prediction_df['port_size'] 

    year_vret = prediction_df.groupby(['date','rank'])['port_ret'].sum()
    year_vret = year_vret.reset_index()
    
    return year_vret

In [5]:
def maximum_return(prediction_df, prediction_col='ret', period='month'):
    
    # Target name based on period
    if period == 'quarter':
        target = 'retq'
    elif period == 'month':
        target = 'ret'
    else:
        raise ValueError("period must be 'quarter' or 'month'")
        
    prediction_df['prob']=prediction_df[prediction_col]
    prediction_df.sort_values('prob', inplace=True)
    prediction_df['rank'] = prediction_df.groupby(['date'])['prob'].transform(lambda x: pd.qcut(x.values, 10, labels=False, duplicates='drop'))
    prediction_df['port_size'] = prediction_df.groupby(['date','rank'])['mve_m'].transform('sum')
    prediction_df['port_ret'] = prediction_df[target] * prediction_df['mve_m']/prediction_df['port_size'] 

    year_vret = prediction_df.groupby(['date','rank'])['port_ret'].sum()
    year_vret = year_vret.reset_index()
    
    return year_vret

In [6]:
def create_result(prediction_parent_path, result_file_name=None, period='month', max_return=False):
    # Get the prediction data paths
    prediction_data_paths = get_prediction_file_paths(prediction_parent_path)
    
    # Postprocess the prediction and append all the results together
    results = pd.DataFrame()
    for df_path in prediction_data_paths:
        df = pd.read_csv(df_path)
        if max_return:
            if period=='month':
                year_vret = maximum_return(df, prediction_col='ret', period='month')
            elif period=='quarter':
                year_vret = maximum_return(df, prediction_col='retq', period='quarter')
            else:
                print('Please input period as either month or quarter')
        else:
            year_vret = postprocess_predictions(df, period=period)
        results = pd.concat([results, year_vret]).reset_index(drop=True)
    
    # Sort the results
    sorted_results = results.sort_values(by=['date', 'rank'],  ascending=[True, True]).reset_index(drop=True)
    
    # Save the sorted results to the same parent directory if file name is given
    if result_file_name:
        sorted_results.to_csv(f'{prediction_parent_path}/{result_file_name}', index=False)
        
    return sorted_results

In [7]:
def compute_returns(path):
    
    # Load data
    df = pd.read_csv(path)

    # Get an average annual  return for each decile
    df_avg = pd.DataFrame(df.groupby('rank')['port_ret'].mean()).reset_index().rename(columns={'port_ret': 'Average of port_ret'})

    # Get the column average for return rates
    df_avg.loc['Return rate']= df_avg.iloc[9] - df_avg.iloc[0]
    df_avg.at['Return rate', 'rank'] = ''
    df_avg = df_avg.fillna('')

    return df_avg

In [8]:
def compute_returns_df(df):
    
    # Get an average annual  return for each decile
    df_avg = pd.DataFrame(df.groupby('rank')['port_ret'].mean()).reset_index().rename(columns={'port_ret': 'Average of port_ret'})

    # Get the column average for return rates
    df_avg.loc['Return rate']= df_avg.iloc[9] - df_avg.iloc[0]
    df_avg.at['Return rate', 'rank'] = ''
    df_avg = df_avg.fillna('')

    return df_avg

### Current Quarterly New Restricted

In [9]:
period = 'quarter'
prediction_parent_path = f'kevin/output/prediction/{period}ly_new_restricted'
result_file_name = 'result.csv'

In [10]:
results = create_result(prediction_parent_path, result_file_name, period, max_return=False)

In [11]:
compute_returns_df(results)

/tmp/ipykernel_3171271/4045607825.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_avg.at['Return rate', 'rank'] = ''


,rank,Average of port_ret
0,0.0,0.013289
1,1.0,0.025109
2,2.0,0.029242
3,3.0,0.031388
4,4.0,0.033673
5,5.0,0.030429
6,6.0,0.034697
7,7.0,0.034552
8,8.0,0.046154
9,9.0,0.054191


### Current Quarterly New Vars

In [12]:
period = 'quarter'
prediction_parent_path = f'kevin/output/prediction/{period}ly_new_vars'
result_file_name = 'result.csv'

In [13]:
results = create_result(prediction_parent_path, result_file_name, period, max_return=False)

In [14]:
compute_returns_df(results)

/tmp/ipykernel_3171271/4045607825.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_avg.at['Return rate', 'rank'] = ''


,rank,Average of port_ret
0,0.0,0.001251
1,1.0,0.015423
2,2.0,0.020356
3,3.0,0.024634
4,4.0,0.023050
5,5.0,0.032089
6,6.0,0.034149
7,7.0,0.033455
8,8.0,0.038631
9,9.0,0.044182


### Current Monthly New Restricted

In [15]:
period = 'month'
prediction_parent_path = f'kevin/output/prediction/{period}ly_new_restricted'
result_file_name = 'result.csv'

In [16]:
results = create_result(prediction_parent_path, result_file_name, period, max_return=False)

In [17]:
compute_returns_df(results)

/tmp/ipykernel_3171271/4045607825.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_avg.at['Return rate', 'rank'] = ''


,rank,Average of port_ret
0,0.0,-0.003345
1,1.0,0.004338
2,2.0,0.007972
3,3.0,0.009160
4,4.0,0.010232
5,5.0,0.012578
6,6.0,0.013128
7,7.0,0.014217
8,8.0,0.015865
9,9.0,0.021817


### Current Monthly New Vars

In [18]:
period = 'month'
prediction_parent_path = f'kevin/output/prediction/{period}ly_new_vars'
result_file_name = 'result.csv'

In [19]:
results = create_result(prediction_parent_path, result_file_name, period, max_return=False)

In [20]:
compute_returns_df(results)

/tmp/ipykernel_3171271/4045607825.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_avg.at['Return rate', 'rank'] = ''


,rank,Average of port_ret
0,0.0,-0.007773
1,1.0,0.003617
2,2.0,0.005597
3,3.0,0.006285
4,4.0,0.008963
5,5.0,0.009865
6,6.0,0.009604
7,7.0,0.011134
8,8.0,0.012588
9,9.0,0.017090


### Monthly Old Vars

In [21]:
compute_returns('kevin/output/prediction/monthly_old_vars/result.csv')

/tmp/ipykernel_3171271/2335035349.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_avg.at['Return rate', 'rank'] = ''


,rank,Average of port_ret
0,0.0,0.003681
1,1.0,0.007112
2,2.0,0.009797
3,3.0,0.009424
4,4.0,0.011221
5,5.0,0.011056
6,6.0,0.011664
7,7.0,0.010712
8,8.0,0.012864
9,9.0,0.017309


### Analyzing result files

In [22]:
def analyze_result_files(result_dir):
    result = pd.read_csv(result_dir)
    result['year'] = result['date'].str[:4]
    summary = pd.pivot_table(data=result, index='year', columns='rank', values='port_ret', aggfunc='mean')
    summary['return'] = summary[9.0] - summary[0.0]

    return summary

In [23]:
quarterly_restricted_result_dir = 'kevin/output/prediction/quarterly_new_restricted/result.csv'
quarterly_new_result_dir = 'kevin/output/prediction/quarterly_new_vars/result.csv'
monthly_restricted_result_dir = 'kevin/output/prediction/monthly_new_restricted/result.csv'
monthly_new_result_dir = 'kevin/output/prediction/monthly_new_vars/result.csv'

### Quarterly New Restricted

In [24]:
analyze_result_files(quarterly_restricted_result_dir)

rank,0,1,2,3,4,5,6,7,8,9,return
year,,,,,,,,,,,
1985,0.053303,0.081361,0.072383,0.075900,0.089916,0.076854,0.078322,0.075168,0.083178,0.070111,0.016807
1986,0.038675,0.049798,0.034040,0.043670,0.044261,0.036239,0.051023,0.034492,0.044243,0.041298,0.002623
1987,0.009976,0.030064,0.011052,0.029276,0.023999,0.016639,0.003107,0.024213,0.016547,0.014126,0.004150
1988,-0.017456,0.026807,0.046970,0.027694,0.048664,0.040783,0.044029,0.039901,0.030149,0.047634,0.065090
1989,0.055127,0.058661,0.059039,0.058313,0.054836,0.072495,0.064798,0.088405,0.064325,0.070516,0.015389
1990,-0.095201,-0.058611,-0.023418,-0.003278,-0.007505,0.009274,-0.006832,0.025546,0.078994,0.121227,0.216428
1991,0.053497,0.109679,0.104737,0.071752,0.074007,0.056903,0.080208,0.081509,0.072164,0.084542,0.031045
1992,0.013868,0.024895,0.019647,0.008368,0.045891,0.037029,0.047281,0.007833,0.019182,0.011895,-0.001972
1993,0.030373,0.005591,0.040451,0.022601,0.030861,0.016112,0.026673,0.020998,0.037648,0.051395,0.021021


### Quarterly New Vars

In [25]:
analyze_result_files(quarterly_new_result_dir)

rank,0,1,2,3,4,5,6,7,8,9,return
year,,,,,,,,,,,
1985,0.030180,0.061176,0.069046,0.071347,0.076493,0.079774,0.070543,0.073199,0.089351,0.079883,0.049703
1986,0.041261,0.049589,0.011044,0.007633,0.030332,0.057351,0.028486,0.053350,0.049367,0.059959,0.018698
1987,-0.016956,0.030868,-0.034000,0.040821,0.006284,0.024170,0.013889,0.023518,0.019959,0.014004,0.030960
1988,0.011644,0.030754,0.032887,0.039786,0.018102,0.040398,0.035793,0.029030,0.048725,0.048619,0.036975
1989,-0.010376,0.049991,0.084268,0.056545,0.073032,0.051978,0.060849,0.070195,0.076608,0.068130,0.078505
1990,-0.094815,-0.077337,-0.015755,-0.035880,-0.022691,-0.032226,-0.003585,-0.014891,-0.004555,0.006942,0.101757
1991,0.122993,0.064163,0.090598,0.063533,0.075504,0.059176,0.067624,0.072366,0.083289,0.088182,-0.034811
1992,0.042259,0.017025,0.047166,-0.010091,-0.019929,0.017907,0.023761,0.034069,0.061427,0.049961,0.007702
1993,0.019272,0.052232,0.027357,0.060156,0.037279,0.034228,0.049998,0.044119,0.035535,0.014374,-0.004897


### Monthly New Restricted

In [26]:
analyze_result_files(monthly_restricted_result_dir)

rank,0,1,2,3,4,5,6,7,8,9,return
year,,,,,,,,,,,
1985,0.007782,0.011813,0.019460,0.017572,0.021380,0.022893,0.023536,0.029384,0.026642,0.025835,0.018053
1986,0.014574,0.011960,0.018578,0.007464,0.017883,0.009836,0.012055,0.012024,0.016417,0.015458,0.000885
1987,0.011090,0.010322,0.006674,0.005815,0.003624,0.000122,0.004672,-0.003979,0.003178,0.003325,-0.007765
1988,0.006600,0.011435,0.014111,0.013456,0.012173,0.013616,0.015319,0.013950,0.018627,0.020710,0.014110
1989,0.012712,0.019225,0.023757,0.018878,0.023465,0.020875,0.024537,0.021647,0.022032,0.027609,0.014897
1990,-0.035582,-0.007281,-0.000781,-0.000133,-0.011434,0.001176,-0.002714,0.000332,0.000488,0.000400,0.035982
1991,0.023212,0.035030,0.030159,0.026895,0.023354,0.021917,0.025314,0.025098,0.028362,0.026614,0.003401
1992,0.000234,0.007593,0.007893,0.009300,0.012536,0.009336,0.010672,0.015046,0.009858,0.019359,0.019124
1993,-0.007379,0.006146,0.008846,0.009134,0.006758,0.006351,0.010807,0.007852,0.008616,0.021636,0.029014


### Monthly New Vars

In [27]:
analyze_result_files(monthly_new_result_dir)

rank,0,1,2,3,4,5,6,7,8,9,return
year,,,,,,,,,,,
1985,0.014331,0.019143,0.020464,0.024560,0.025358,0.022833,0.025262,0.025680,0.021550,0.025774,0.011442
1986,0.014398,0.014759,0.008522,0.003413,0.011545,0.011044,0.018869,0.016474,0.017197,0.015056,0.000658
1987,-0.013855,0.006719,-0.008389,0.006442,0.006450,0.005321,0.005207,0.008420,0.006640,0.006213,0.020069
1988,0.008492,0.013399,0.012818,0.012006,0.013945,0.011031,0.017347,0.016781,0.014672,0.018569,0.010077
1989,-0.010562,0.010132,0.024954,0.010801,0.019342,0.014416,0.020380,0.018724,0.023956,0.027051,0.037612
1990,-0.051837,-0.029941,-0.030152,-0.023404,-0.018250,-0.008404,-0.003326,-0.003937,0.001223,0.001727,0.053563
1991,0.006516,0.028923,0.029407,0.035213,0.032557,0.031931,0.018459,0.025377,0.026336,0.028862,0.022346
1992,0.000163,-0.007815,0.001491,0.008339,-0.004135,0.003358,0.008201,0.011757,0.017935,0.014567,0.014404
1993,-0.008558,0.009024,0.007963,0.000865,0.007726,0.006136,0.005867,0.007832,0.011467,0.015079,0.023637


### RF results reference

In [28]:
quarterly_restricted_result_dir_RF = 'Info Processing and Mutual Funds/new_input/results/result_w_c_q_new_all_restrict_95_stocks.csv'
quarterly_restricted_result_dir_RF = 'Info Processing and Mutual Funds/new_input/results/result_w_c_q_new_all_restrict_95.csv'

In [29]:
analyze_result_files(quarterly_restricted_result_dir_RF)

rank,0,1,2,3,4,5,6,7,8,9,return
year,,,,,,,,,,,
1986,-0.042276,0.000715,0.029379,0.030065,0.037146,0.052625,0.051450,0.055685,0.061998,0.054150,0.096426
1987,-0.047072,-0.034725,-0.020583,-0.014037,-0.004612,0.013922,0.007724,0.015038,0.038979,0.057305,0.104377
1988,-0.032986,-0.013332,0.016615,0.027845,0.039641,0.027500,0.036896,0.029400,0.062737,0.069541,0.102527
1989,-0.039194,0.001860,0.021272,0.041653,0.046331,0.039573,0.048253,0.069729,0.081896,0.093934,0.133128
1990,-0.180627,-0.135499,-0.088966,-0.049471,-0.040899,-0.012314,-0.025692,-0.014494,-0.000844,0.008721,0.189348
1991,0.084129,0.082954,0.050934,0.059230,0.049883,0.066490,0.068891,0.083475,0.116456,0.162577,0.078448
1992,-0.041512,0.009157,0.014145,0.038007,0.030571,0.029338,0.023477,0.024085,0.005520,-0.010597,0.030915
1993,-0.015621,0.022742,0.016251,0.047191,0.018877,0.038794,0.055268,0.062425,0.061826,0.051368,0.066990
1994,-0.107817,-0.054129,-0.010377,0.004112,0.000562,0.006738,-0.000606,0.007396,-0.005335,0.043673,0.151490


In [30]:
compute_returns(quarterly_restricted_result_dir_RF)

/tmp/ipykernel_3171271/2335035349.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_avg.at['Return rate', 'rank'] = ''


,rank,Average of port_ret
0,0.0,-0.035549
1,1.0,-0.002851
2,2.0,0.012681
3,3.0,0.026528
4,4.0,0.030880
5,5.0,0.031569
6,6.0,0.039013
7,7.0,0.044643
8,8.0,0.047770
9,9.0,0.066568
